## Part 1

In [3]:
import findspark
findspark.init()
findspark.find()

'C:\\Spark\\spark-hadoop'

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]")\
.config("spark.driver.memory", "28g")\
.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11")\
.appName("ch31_DeepLearningLappy-docs").getOrCreate()

#.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
# .config("spark.driver.memory", "8g")\
# .config("spark.executor.memory", "3g")\
# .config("spark.executor.cores", "4")\
# .config("spark.executor.instances", "10")\


# This file is sourced when running various Spark programs.
# Copy it as spark-env.sh and edit that to configure Spark for your site.
# Options read in YARN client mode
#SPARK_EXECUTOR_INSTANCES="2" #Number of workers to start (Default: 2)
#SPARK_EXECUTOR_CORES="1" #Number of cores for the workers (Default: 1).
#SPARK_EXECUTOR_MEMORY="1G" #Memory per Worker (e.g. 1000M, 2G) (Default: 1G)
#SPARK_DRIVER_MEMORY="512M" #Memory for Master (e.g. 1000M, 2G) (Default: 512 Mb)

In [5]:
sc = spark.sparkContext

In [6]:
spark

In [9]:
from sparkdl.image import imageIO
from pyspark.ml.image import ImageSchema

Using TensorFlow backend.


In [12]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import expr
from pyspark.sql.types import DoubleType

In [13]:
# In[15]:
img_dir = "D:/Learn/GitRepos/Spark-The-Definitive-Guide/data/deep-learning-images"

In [14]:
tulips_df = ImageSchema.readImages(img_dir + "/tulips").withColumn("label", lit(1))
tulips_df.count()

30

In [15]:
# In[16]:
daisy_df = imageIO.readImagesWithCustomFn(img_dir + "/daisy", decode_f=imageIO.PIL_decode).withColumn("label", lit(0))
daisy_df.count()

30

In [16]:
# In[ ]:
tulips_train, tulips_test = tulips_df.randomSplit([0.6, 0.4])
daisy_train, daisy_test = daisy_df.randomSplit([0.6, 0.4])

In [17]:
# In[ ]:
train_images_df = tulips_train.unionAll(daisy_train).cache()
test_images_df = tulips_test.unionAll(daisy_test).cache()

In [29]:
print(train_images_df.count())
print(test_images_df.count())

30
30


## Part 2 

In [20]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer

featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="InceptionV3")

lr = LogisticRegression(maxIter=20, regParam=0.05, elasticNetParam=0.3, labelCol="label")

p = Pipeline(stages=[featurizer, lr])

model = p.fit(train_images_df)    # train_images_df is a dataset of images and labels

In [21]:
# Inspect training error
df = model.transform(train_images_df.limit(10))

In [22]:
df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)
 |-- label: integer (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [23]:
predictionAndLabels = df.select("prediction", "label")

In [24]:
predictionAndLabels.count()

10

In [25]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Training set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Training set accuracy = 1.0


In [27]:
# Inspect training error
df = model.transform(test_images_df.limit(10))
predictionAndLabels = df.select("prediction", "label")
print(predictionAndLabels.count())
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

10
Test set accuracy = 1.0


## Part 3

In [80]:
import keras
from keras.applications import InceptionV3

model = InceptionV3(weights="imagenet")
model.save("D:/tmp/model-full-tmp.h5")

In [100]:
import PIL.Image
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from sparkdl.estimators.keras_image_file_estimator import KerasImageFileEstimator

In [101]:
def load_image_from_uri(local_uri):
  img = (PIL.Image.open(local_uri).convert('RGB').resize((299, 299), PIL.Image.ANTIALIAS))
  img_arr = np.array(img).astype(np.float32)
  img_tnsr = preprocess_input(img_arr[np.newaxis, :])
  return img_tnsr

In [102]:
estimator = KerasImageFileEstimator( inputCol="uri",
                                     outputCol="prediction",
                                     labelCol="one_hot_label",
                                     imageLoader=load_image_from_uri,
                                     kerasOptimizer='adam',
                                     kerasLoss='categorical_crossentropy', # why categorical_crossentropy
                                     modelFile="D:/tmp/model-full-tmp.h5" # local file path for model
                                   ) 

In [103]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [104]:
paramGrid = (
  ParamGridBuilder()
  .addGrid(estimator.kerasFitParams, [{"batch_size": 32, "verbose": 0},
                                      {"batch_size": 64, "verbose": 0}])
  .build()
)

In [105]:
bc = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label" )
cv = CrossValidator(estimator=estimator, estimatorParamMaps=paramGrid, evaluator=bc, numFolds=2)

In [106]:
import pyspark.ml.linalg as spla
import pyspark.sql.types as sptyp

import os
#from os import listdir

In [107]:
def CreateTrainImageUriandLabels(image_uris, label, cardinality):
  # Create image categorical labels (integer IDs)
  local_rows = []
  for uri in image_uris:
    label_inds = np.zeros(cardinality)
    label_inds[label] = 1.0
    one_hot_vec = spla.Vectors.dense(label_inds.tolist())
    _row_struct = {"uri": uri, "one_hot_label": one_hot_vec, "label": float(label)}
    row = sptyp.Row(**_row_struct)
    local_rows.append(row)

  image_uri_df = spark.createDataFrame(local_rows)
  return image_uri_df

  

In [108]:
def absoluteFilePaths(directory):
   for dirpath,_,filenames in os.walk(directory):
       for f in filenames:
           yield os.path.abspath(os.path.join(dirpath, f))

In [109]:
absoluteFilePaths(img_dir + "/tulips")

<generator object absoluteFilePaths at 0x0000024E3C1592B0>

In [110]:
label_cardinality = 2

tulips_files = [str(f) for f in absoluteFilePaths(img_dir + "/tulips")]  # make "local" file paths for images
tulips_uri_df = CreateTrainImageUriandLabels(tulips_files,1,label_cardinality)
daisy_files = [str(f) for f in absoluteFilePaths(img_dir + "/daisy")]  # make "local" file paths for images
daisy_uri_df = CreateTrainImageUriandLabels(daisy_files,0,label_cardinality)

In [111]:
#tulips_train, tulips_test, _ = tulips_uri_df.randomSplit([0.005, 0.005, 0.99]) 
# use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)
tulips_train, tulips_test = tulips_uri_df.randomSplit([0.8, 0.2]) 

In [112]:
#daisy_train, daisy_test, _ = daisy_uri_df.randomSplit([0.005, 0.005, 0.99])    
# use larger training sets (e.g. [0.6, 0.4] for non-community edition clusters)
daisy_train, daisy_test = daisy_uri_df.randomSplit([0.8, 0.2]) 

In [113]:
train_df = tulips_train.unionAll(daisy_train).cache()
test_df = tulips_test.unionAll(daisy_test).cache()

In [114]:
train_df.show(10)

+-----+-------------+--------------------+
|label|one_hot_label|                 uri|
+-----+-------------+--------------------+
|  1.0|    [0.0,1.0]|D:\Learn\GitRepos...|
|  1.0|    [0.0,1.0]|D:\Learn\GitRepos...|
|  1.0|    [0.0,1.0]|D:\Learn\GitRepos...|
|  1.0|    [0.0,1.0]|D:\Learn\GitRepos...|
|  1.0|    [0.0,1.0]|D:\Learn\GitRepos...|
|  1.0|    [0.0,1.0]|D:\Learn\GitRepos...|
|  1.0|    [0.0,1.0]|D:\Learn\GitRepos...|
|  1.0|    [0.0,1.0]|D:\Learn\GitRepos...|
|  1.0|    [0.0,1.0]|D:\Learn\GitRepos...|
|  1.0|    [0.0,1.0]|D:\Learn\GitRepos...|
+-----+-------------+--------------------+
only showing top 10 rows



In [115]:
# Under the hood, each of the partitions is fully loaded in memory, which may be expensive.
# This ensure that each of the paritions has a small size.
# train_df = train_df.repartition(100)
# test_df = test_df.repartition(100)

In [117]:
# cvModel = cv.fit(train_df)

# #
# File "d:\learn\.virtualenvs\pyspark-lab\lib\site-packages\keras\engine\training_utils.py", line 137, in standardize_input_data
#     str(data_shape))
# ValueError: Error when checking target: expected predictions to have shape (1000,) but got array with shape (2,)


## part 4

In [ ]:
from pyspark.ml.image import ImageSchema
from sparkdl import DeepImagePredictor

image_df = ImageSchema.readImages(sample_img_dir)

predictor = DeepImagePredictor(inputCol="image", 
                               outputCol="predicted_labels", 
                               modelName="InceptionV3", 
                               decodePredictions=True, 
                               topK=10)

predictions_df = predictor.transform(image_df)

## part 5

In [ ]:
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img
import numpy as np
import os
from pyspark.sql.types import StringType
from sparkdl import KerasImageFileTransformer

def loadAndPreprocessKerasInceptionV3(uri):
  # this is a typical way to load and prep images in keras
  image = img_to_array(load_img(uri, target_size=(299, 299)))  # image dimensions for InceptionV3
  image = np.expand_dims(image, axis=0)
  return preprocess_input(image)

transformer = KerasImageFileTransformer(inputCol="uri", outputCol="predictions",
                                        modelFile='/tmp/model-full-tmp.h5',  # local file path for model
                                        imageLoader=loadAndPreprocessKerasInceptionV3,
                                        outputMode="vector")

files = [os.path.abspath(os.path.join(dirpath, f)) for f in os.listdir("/data/myimages") if f.endswith('.jpg')]
uri_df = sqlContext.createDataFrame(files, StringType()).toDF("uri")

keras_pred_df = transformer.transform(uri_df)

## Part 6

In [ ]:
from sparkdl import KerasTransformer
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

# Generate random input data
num_features = 10
num_examples = 100
input_data = [{"features" : np.random.randn(num_features).tolist()} for i in range(num_examples)]
input_df = sqlContext.createDataFrame(input_data)

# Create and save a single-hidden-layer Keras model for binary classification
# NOTE: In a typical workflow, we'd train the model before exporting it to disk,
# but we skip that step here for brevity
model = Sequential()
model.add(Dense(units=20, input_shape=[num_features], activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))
model_path = "/tmp/simple-binary-classification"
model.save(model_path)

# Create transformer and apply it to our input data
transformer = KerasTransformer(inputCol="features", outputCol="predictions", modelFile=model_path)
final_df = transformer.transform(input_df)

## Part 7

In [ ]:
from keras.applications import InceptionV3
from sparkdl.udf.keras_image_model import registerKerasImageUDF

registerKerasImageUDF("inceptionV3_udf", InceptionV3(weights="imagenet"))

In [ ]:
registerKerasImageUDF("my_custom_keras_model_udf", "/tmp/model-full-tmp.h5")

In [ ]:
from keras.applications import InceptionV3
from sparkdl.udf.keras_image_model import registerKerasImageUDF

def keras_load_img(fpath):
    from keras.preprocessing.image import load_img, img_to_array
    import numpy as np
    img = load_img(fpath, target_size=(299, 299))
    return img_to_array(img).astype(np.uint8)

registerKerasImageUDF("inceptionV3_udf_with_preprocessing", InceptionV3(weights="imagenet"), keras_load_img)

In [ ]:
from pyspark.ml.image import ImageSchema

image_df = ImageSchema.readImages(sample_img_dir)
image_df.registerTempTable("sample_images")

In [ ]:
SELECT my_custom_keras_model_udf(image) as predictions from sample_images